In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!pip install langchain-community pypdf sentence_transformers faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

In [5]:
# importing necessary libraries

from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters.character import RecursiveCharacterTextSplitter
import os
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np


main_docs = []


files_path = "/content/drive/MyDrive/Captures/data/data"

if os.path.exists(files_path):
  for folder in os.listdir(files_path):
    print("in folder:", folder)
    for file in os.listdir(files_path+'/'+folder):
      loader = PyPDFLoader(files_path+'/'+folder+'/'+file)
      documents = loader.load()
      for i in range(1, len(documents)):
        documents[0].page_content += documents[i].page_content

      documents = [documents[0]]
      text_splitter = RecursiveCharacterTextSplitter(
          chunk_size=1000,
          chunk_overlap=200,
          length_function=len
      )
      docs = text_splitter.split_documents(documents)
      main_docs.extend(docs)


embeddings = SentenceTransformer("sentence-transformers/all-MiniLM-L12-v2")


def create_faiss_index(docs, embeddings):
  """Creates a FAISS index from a list of documents and an embedding model.

  Args:
    docs: A list of documents.
    embeddings: A SentenceTransformer model for generating embeddings.

  Returns:
    A FAISS index.
  """

  # generating embeddings for the documents
  doc_embeddings = embeddings.encode([doc.page_content for doc in docs])

  # creating a FAISS index
  d = doc_embeddings.shape[1]

  nlist = 10  # hno of cells
  quantizer = faiss.IndexFlatL2(d)
  index = faiss.IndexIVFFlat(quantizer, d, nlist)
  index.train(doc_embeddings)
  index.add(doc_embeddings)

  return index

#  FAISS index
index = create_faiss_index(main_docs, embeddings)

# Saving the FAISS index to a file
faiss.write_index(index, "faiss_index.index")



in folder: ENGINEERING
in folder: SALES
in folder: FITNESS
in folder: INFORMATION-TECHNOLOGY
in folder: HEALTHCARE
in folder: PUBLIC-RELATIONS
in folder: DIGITAL-MEDIA
in folder: HR
in folder: TEACHER
in folder: FINANCE
in folder: ARTS
in folder: AUTOMOBILE
in folder: AVIATION
in folder: CONSULTANT
in folder: CONSTRUCTION
in folder: BUSINESS-DEVELOPMENT
in folder: CHEF
in folder: DESIGNER
in folder: BPO
in folder: BANKING
in folder: ADVOCATE
in folder: ACCOUNTANT
in folder: AGRICULTURE
in folder: APPAREL


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
# loading the index and retrieving the top 10 profiles

index = faiss.read_index("faiss_index.index")


job_description = """
We are looking for a skilled UI Developer to join our dynamic team.
The ideal candidate will have a strong background in front-end development,
with proficiency in HTML, CSS, JavaScript, and modern frameworks like React or Angular.
Your primary responsibility will be to create visually appealing and user-friendly web interfaces that enhance user experience and align with our brand guidelines
"""

# input query vector embedding
xq = embeddings.encode([job_description])
D, I = index.search(xq, k=10)  # search


# Accessing multiple documents using list comprehension
result_docs = [main_docs[i] for i in I[0]]
for doc in result_docs:
  print(doc.metadata['source'])

/content/drive/MyDrive/Captures/data/data/INFORMATION-TECHNOLOGY/83816738.pdf
/content/drive/MyDrive/Captures/data/data/HR/18297650.pdf
/content/drive/MyDrive/Captures/data/data/ENGINEERING/28630325.pdf
/content/drive/MyDrive/Captures/data/data/DESIGNER/37664296.pdf
/content/drive/MyDrive/Captures/data/data/HR/18297650.pdf
/content/drive/MyDrive/Captures/data/data/INFORMATION-TECHNOLOGY/46260230.pdf
/content/drive/MyDrive/Captures/data/data/BANKING/71422121.pdf
/content/drive/MyDrive/Captures/data/data/HR/18297650.pdf
/content/drive/MyDrive/Captures/data/data/INFORMATION-TECHNOLOGY/13405733.pdf
/content/drive/MyDrive/Captures/data/data/INFORMATION-TECHNOLOGY/16899268.pdf
